# COVID-19 data reader

Questo notebook carica i dati regionali riguardanti la diffusione e la gestione del COVID-19 direttamente dal repository delle Protezione civile.

I dati sono così organizzati:
- per ciascuna feature si crea un dizionario i cui elementi sono le regioni, accessibili per nome (es. `trend[indicatore][regione]`)
- ogni elemento contiene il valore misurato nei giorni di osservazione
- le date sono nella lista di stringhe `giorni`
- è stata creata una entry per ciascun dizionario dell'indicatore per l'*Italia*. Questo indicatore, per quanto riguarda gli indicatori di base originali, ne è la somma, sltrimenti è opportunamente calcolato.


Fonte dati: *Presidenza del Consiglio dei Ministri - Dipartimento della Protezione Civile*

https://github.com/pcm-dpc/COVID-19

> **NOTA** Questo notebook nasce per condividere il proprio kernel con altri notebooks dove vengono effettuate e visualizzate le analisi vere e proprie.

## Lista indicatori raccolti nella struttura dati

**Originali:**
- ricoverati_con_sintomi
- terapia_intensiva
- totale_ospedalizzati
- isolamento_domiciliare
- totale_positivi
- variazione_totale_positivi
- nuovi_positivi
- dimessi_guariti
- deceduti
- totale_casi
- tamponi

**Derivati:**
- tamponi_giornalieri
- perc_tamponi_positivi

**Altre variabili:**
- giorni
- day_counter
- regioni

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Parametri

In [2]:
#data_file='dpc-covid19-ita-regioni.csv' # locale non aggiornato
data_file='https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

## Lettura dati e preprocessing

In [3]:
data=pd.read_csv(data_file)
print(data.columns)

giorni=pd.unique(data['data'])
day_counter=list(range(len(giorni)))
print('Giorni osservati: %d'%len(giorni))
print('Da: %s'%giorni[0][0:10])
print(' A: %s'%giorni[-1][0:10])
regioni=pd.unique(data['denominazione_regione'])

data[data['denominazione_regione']=='Lombardia']


Index(['data', 'stato', 'codice_regione', 'denominazione_regione', 'lat',
       'long', 'ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi', 'note_it', 'note_en'],
      dtype='object')
Giorni osservati: 42
Da: 2020-02-24
 A: 2020-04-05


,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,note_it,note_en
9,2020-02-24T18:00:00,ITA,3,Lombardia,45.466794,9.190347,76,19,95,71,166,0,166,0,6,172,1463,NaN,NaN
30,2020-02-25T18:00:00,ITA,3,Lombardia,45.466794,9.190347,79,25,104,127,231,65,68,0,9,240,3700,NaN,NaN
51,2020-02-26T18:00:00,ITA,3,Lombardia,45.466794,9.190347,79,25,104,145,249,18,18,0,9,258,3208,NaN,NaN
72,2020-02-27T18:00:00,ITA,3,Lombardia,45.466794,9.190347,172,41,213,136,349,100,145,40,14,403,3320,NaN,NaN
93,2020-02-28T18:00:00,ITA,3,Lombardia,45.466794,9.190347,235,47,282,192,474,125,128,40,17,531,4835,NaN,NaN
114,2020-02-29T17:00:00,ITA,3,Lombardia,45.466794,9.190347,256,80,336,216,552,78,84,40,23,615,5723,NaN,NaN
135,2020-03-01T17:00:00,ITA,3,Lombardia,45.466794,9.190347,406,106,512,375,887,335,369,73,24,984,6879,NaN,NaN
156,2020-03-02T18:00:00,ITA,3,Lombardia,45.466794,9.190347,478,127,605,472,1077,190,270,139,38,1254,7925,NaN,NaN
177,2020-03-03T18:00:00,ITA,3,Lombardia,45.466794,9.190347,698,167,865,461,1326,249,266,139,55,1520,9577,NaN,NaN
198,2020-03-04T17:00:00,ITA,3,Lombardia,45.466794,9.190347,877,209,1086,411,1497,171,300,250,73,1820,12138,NaN,NaN


## Analisi

### Creazione della struttura dati

- Si crea un dizionario per ciascuno degli indicatori selezionati
- a sua volta ciascun elemento del primo dizionario è un dizionario con le regioni come chiave
    - ogni elemento indicatore contiene i valori dell'indicatore per la regione

**Esempio:**
```    
trend[indicatore][regione]
```

In [4]:
# indicatori osservati
indicatori=['ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi']

# qui immetto tutti i trend per ciascuna regione
trend={}
for ind in indicatori:
    trend[ind]={}
    for regione in regioni:
        trend[ind][regione]=np.array(data[data['denominazione_regione']==regione][ind])

# calcolo gli indicatori nazionali come somma delle regioni
for ind in indicatori:
    trend[ind]['Italia']=np.zeros(len(giorni))
    for regione in regioni:
        trend[ind]['Italia']+=trend[ind][regione]

### Indicatori derivati

#### Tamponi giornalieri

In [5]:
# tamponi giornalieri
trend['tamponi_giornalieri']={}
for regione in regioni:
    trend['tamponi_giornalieri'][regione]=np.insert(np.diff(trend['tamponi'][regione]),0,0)

trend['tamponi_giornalieri']['Italia']=np.insert(np.diff(trend['tamponi']['Italia']),0,0)


#### Percentuale tamponi positivi

In [6]:
# percentuale tamponi positivi su quelli effettuati
trend['perc_tamponi_positivi']={}
for regione in regioni:
    trend['perc_tamponi_positivi'][regione]=100*trend['nuovi_positivi'][regione]/trend['tamponi_giornalieri'][regione]

trend['perc_tamponi_positivi']['Italia']=100*trend['nuovi_positivi']['Italia']/trend['tamponi_giornalieri']['Italia']